https://www.rareskills.io/post/bilinear-pairing

The chances that

`e(aG, bG′) = e(abG, G′) = e(G,abG′)`

is true for three random groups is very slim.


Using groups of different “dimensions,” plus some other features which are far more complex than we want to get into here, make this property easier to construct.

Remember, this is a cyclic group. You don’t care what bizzare moon-math object you are “multiplying” by. You just care that it follows the properties of cyclic groups. Then you can leave the under-the-hood details to a library, which we will discuss next.

So don’t let the name “field extension” scare you. All that matters to us is that we will be dealing with elliptic curve points that have more dimensions than our typical (x, y) point on the curve.


Specifically, in the next section we will be dealing with three groups: G1, G2, and G12. These correspond to the dimensions of these groups.

In [2]:
from py_ecc.bn128 import G1, G2, pairing, add, multiply, eq

print(G1)

print(G2)


(1, 2)
((10857046999023057135944570762232829481370756359578518086990519993285655852781, 11559732032986387107991004021392285783925812861821192530917403151452391805634), (8495653923123431417604973247489272438418190587263600148770280649306958101930, 4082367875863433681332203403145435568316851327593401208105741076214120093531))


In [8]:
print(eq(add(G1, G1), multiply(G1, 2)))
# True
print(eq(add(G2, G2), multiply(G2, 2)))
# True

True
True


In [1]:
import sys 
sys.setrecursionlimit(20000)

A = multiply(G2, 5) # 5G2
B = multiply(G1, 6) # 6G1
pairing(A, B)

C = multiply(G2, 5 * 6) # 30G2

pairing(A, B) == pairing(C, G1) # (AG2, BG1) == (ABG2, G1)

NameError: name 'multiply' is not defined

So if we want to be mathematical about this, our bn128 pairing does the following

`e: G1 × G2 → G12`

In [6]:
pairing(G2 * 5, G1 * 6) == pairing(G2 * 30, G1) # (5G2, 6G1) == (30G2, G1)


ValueError: too many values to unpack (expected 2)

## Bilinear Pairings in Ethereum
### EIP 197 Specs

The ethereum Precompile defined in EIP-197 works on points in G1 and G2 and **implicitly** works on points in G12

The specification takes in a list of G1 and G2 points laid out as:

`A₁B₁A₂B₂...AₙBₙ : Aᵢ ∈ G1, Bᵢ ∈ G2`

These were created as

`A₁ = a₁G1 ; B₁ = b₁G2 ; A₂ = a₂G1 ; B₂ = b₂G2 ; ... ; Aₙ = aₙG1 ; Bₙ = bₙG2`

The precompile returns 1 if the following is true

`a₁b₁ + a₂b₂ + ... + aₙbₙ = 0`

and 0 otherwise

### Justification for EIP 197 design decision
Adding a wrapped around `e: G1 × G2 → G12` is that problem because G12 points are huge 
-> Take up a lot of space in memory -> larger gas cost

And most zk verification aglorithms don't check the value of the output of a pairing but only that it is equal to other pairings.

Final step of Groth16 (ZK by tornado cash) looks like

`e(A₁, B₂) = e(α₁, β₂) + e(L₁, γ₂) + e(C₁, δ₂)`

where each variable is an elliptic curve point of either G1 or G2 based on its subscript notation

We can write it as the following to match the precompile specification

`0 = e(−A₁, B₂) + e(α₁, β₂) + e(L₁, γ₂) + e(C₁, δ₂)`

### Sum of preimages
If
`ab + cd = 0` 

then it must be true in G12 space
`A₁B₂ + C₁D₂ = 0₁₂  A₁,C₁ ∈ G1, B₂,D₂ ∈ G2`

The precompile isn't computing the discrete logarithm (`a₁b₂ + c₁d₂ = 0`), it's simply checking if the sum of pairings is zero

### -ab + cd = 0 Example
`a = 4 ; b = 3 ; c = 6 ; d = 2`

putting it into the formula we can get

`A₁B₂ + C₁D₂  = e(−aG1, bG2) + e(cG1, dG2) = 0`

In [2]:
from py_ecc.bn128 import neg, multiply, G1, G2
a = 4
b = 3
c = 6
d = 2
# negate G1 * a to make the equation sum up to 0

print(neg(multiply(G1, a)))
#(3010198690406615200373504922352659861758983907867017329644089018310584441462, 17861058253836152797273815394432013122766662423622084931972383889279925210507)
print(multiply(G2, b))

# (
#(2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653)
#, 
#(2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866)
#)

print(multiply(G1, c))
# (4503322228978077916651710446042370109107355802721800704639343137502100212473, 6132642251294427119375180147349983541569387941788025780665104001559216576968)
print(multiply(G2, d))
# (
#(18029695676650738226693292988307914797657423701064905010927197838374790804409, 14583779054894525174450323658765874724019480979794335525732096752006891875705),
#(2140229616977736810657479771656733941598412651537078903776637920509952744750, 11474861747383700316476719153975578001603231366361248090558603872215261634898)
#)

(3010198690406615200373504922352659861758983907867017329644089018310584441462, 17861058253836152797273815394432013122766662423622084931972383889279925210507)
((2725019753478801796453339367788033689375851816420509565303521482350756874229, 7273165102799931111715871471550377909735733521218303035754523677688038059653), (2512659008974376214222774206987427162027254181373325676825515531566330959255, 957874124722006818841961785324909313781880061366718538693995380805373202866))
(4503322228978077916651710446042370109107355802721800704639343137502100212473, 6132642251294427119375180147349983541569387941788025780665104001559216576968)
((18029695676650738226693292988307914797657423701064905010927197838374790804409, 14583779054894525174450323658765874724019480979794335525732096752006891875705), (2140229616977736810657479771656733941598412651537078903776637920509952744750, 11474861747383700316476719153975578001603231366361248090558603872215261634898))


Now we have the values encrypted into points on G1 and G2 groups, someone else or a program can confirm that we computed `A1B2+C1D2=0` correctly without knowing the individual values of a, b, c, or d. Here’s a solidity contract that uses the ecPairing precompile to confirm that we computed the equations with valid values.